# Imports

In [ ]:
import requests
import json
import pandas as pd
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import io
import csv
import time

# Configurações da API

In [ ]:
# Configurações
BASE_URL = "http://api.worldbank.org/v2/country"
PER_PAGE = "20000"

# Dicionário de indicadores
INDICATORS = {
    "dimen_intrnt_users": "IT.NET.USER.ZS",
    "dimen_fixed_broadband_users": "IT.NET.BBND.P2",
    "dimen_cellphone_users": "IT.CEL.SETS.P2",
    "dimen_secure_internet_servers": "IT.NET.SECR.P6",
    "dimen_tic_export": "BX.GSR.CCIS.CD",
    "dimen_cntry_pop": "SP.POP.TOTL",
    "dimen_cntry_pib": "NY.GDP.PCAP.CD",
    "dimen_mrkt_stck_rtrn": "GFDD.OM.02"
}

# Funções

In [ ]:
def get_world_bank_countries():
    countries_data = []
    base_url = "https://api.worldbank.org/v2/country"
    params = {
        "format": "json",
        "per_page": 300
    }

    try:
        response = requests.get(base_url, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        current_countries = data

        for country in current_countries:
            processed_country = {
                "country_id": country.get("id"),
                "country_name": country.get("name"),
                "region_id": country.get("region", {}).get("id"),
                "region_value": country.get("region", {}).get("value"),
                "admin_region_id": country.get("adminregion", {}).get("id"),
                "admin_region_value": country.get("adminregion", {}).get("value"),
                "income_level_id": country.get("incomeLevel", {}).get("id"),
                "income_level_value": country.get("incomeLevel", {}).get("value") ,
                "lending_type_id": country.get("lendingType", {}).get("id") ,
                "lending_type_value": country.get("lendingType", {}).get("value"),
                "capital_city": country.get("capitalCity"),
                "longitude": country.get("longitude"),
                "latitude": country.get("latitude")
            }
            countries_data.append(processed_country)

    except requests.exceptions.RequestException as e:
        print(f" -> Erro ao buscar dados da API do Banco Mundial: {e}")
        return []
    except ValueError as e:
        print(f" -> Erro ao decodificar JSON da API: {e}")
        return []
    except Exception as e:
        print(f" -> Um erro inesperado ocorreu ao buscar países: {e}")
        return []

    print(f"Total de {len(countries_data)} informações de países/regiões agregadas encontradas na primeira página.")
    return countries_data


def fetch_indicator_data(indicator_code, indicator_name_friendly):
    url = f"{BASE_URL}/all/indicator/{indicator_code}"
    params = {
        "format": "json",
        "per_page": PER_PAGE
    }
    print(f"  Buscando: {indicator_name_friendly} ({indicator_code})... URL: {requests.Request('GET', url, params=params).prepare().url}")

    try:
        response = requests.get(url, params=params, timeout=45)
        response.raise_for_status()
        data = response.json()

        if not data or len(data) < 2 or data[1] is None:
            print(f"  -> Dados não encontrados ou formato inesperado para o indicador {indicator_code}.")
            return []

        processed_data = []
        for record in data[1]:
            processed_data.append({
                "year": record.get('date'),
                "ind_value": record.get('value'),
                "ind_id": record.get('indicator', {}).get('id'),
                "ind_name": record.get('indicator', {}).get('value'),
                "cntry_id": record.get('countryiso3code'),
                "cntry_name": record.get('country', {}).get('value')
            })

        processed_data.sort(key=lambda x: x['year'])
        if processed_data:
            print(f"  -> {len(processed_data)} registros encontrados para {indicator_name_friendly}.")
        else:
            print(f"  -> Nenhum registro com valor encontrado para {indicator_name_friendly}.")
        return processed_data

    except requests.exceptions.Timeout:
        print(f"  Erro de Timeout ao buscar dados para {indicator_code}.")
        return []
    except requests.exceptions.RequestException as e:
        print(f"  Erro na requisição para {indicator_code}: {e}")
        return []
    except ValueError as e:
        print(f"  Erro ao decodificar JSON para {indicator_code}: {e}")
        return []

def save_data_to_bigquery_table(data,
                                table_name,
                                dataset_id="raw",
                                project_id="learned-tube-459813-c5",
                                write_disposition="WRITE_TRUNCATE"):
    if not data:
        print(" -> Nenhum dado fornecido para salvar no BigQuery.")
        return

    print(f"Iniciando carregamento para BigQuery: {project_id or 'PROJETO_PADRAO'}.{dataset_id}.{table_name}...")

    try:
        df = pd.DataFrame(data)

        if df.empty:
            print(f" -> DataFrame vazio após conversão dos dados para a tabela '{table_name}'. Nada será salvo.")
            return

        client = bigquery.Client(project=project_id)

        if project_id:
            table_id = f"{project_id}.{dataset_id}.{table_name}"
        else:
            table_id = f"{client.project}.{dataset_id}.{table_name}"
            print(f"    (Usando projeto inferido: {client.project})")


        job_config = bigquery.LoadJobConfig(
            autodetect=True,
            write_disposition=write_disposition,
        )

        job = client.load_table_from_dataframe(
            df, table_id, job_config=job_config
        )
        job.result()

        table = client.get_table(table_id)
        print(
            f" -> Dados carregados com sucesso para a tabela '{table_id}'. "
            f"Total de linhas: {table.num_rows}"
        )

    except ImportError:
        print(" -> Erro de importação: Verifique se pandas e google-cloud-bigquery estão instalados.")
    except NotFound as e:
        print(f" -> Erro: Dataset '{dataset_id}' (ou projeto) não encontrado ou sem permissão: {e}")
    except Exception as e:
        print(f" -> Um erro inesperado ocorreu ao carregar dados para '{table_name}' no BigQuery: {e}")


# Execução

In [ ]:
if __name__ == "__main__":
    total_indicators = len(INDICATORS)

    countries_data = get_world_bank_countries()
    save_data_to_bigquery_table(countries_data,"dimen_info_countries")

    print("-" * 40)

    for i, (friendly_name, indicator_code) in enumerate(INDICATORS.items()):
        print(f"Processando indicador {i+1} de {total_indicators}: {friendly_name}")
        indicator_data = fetch_indicator_data(indicator_code, friendly_name)
        if indicator_data:
            save_data_to_bigquery_table(indicator_data, friendly_name)

        if i < total_indicators - 1:
            time.sleep(1)
        print("-" * 40)

 -> Processando 296 registros da página 1 (total de páginas na API: 1).
Total de 296 informações de países/regiões agregadas encontradas na primeira página.
Iniciando carregamento para BigQuery: learned-tube-459813-c5.raw.dimen_info_countries...
 -> Dados carregados com sucesso para a tabela 'learned-tube-459813-c5.raw.dimen_info_countries'. Total de linhas: 296
----------------------------------------
Processando indicador 1 de 8: dimen_intrnt_users
  Buscando: dimen_intrnt_users (IT.NET.USER.ZS)... URL: http://api.worldbank.org/v2/country/all/indicator/IT.NET.USER.ZS?format=json&per_page=20000
  -> 17290 registros encontrados para dimen_intrnt_users.
Iniciando carregamento para BigQuery: learned-tube-459813-c5.raw.dimen_intrnt_users...
 -> Dados carregados com sucesso para a tabela 'learned-tube-459813-c5.raw.dimen_intrnt_users'. Total de linhas: 17290
----------------------------------------
Processando indicador 2 de 8: dimen_fixed_broadband_users
  Buscando: dimen_fixed_broadband_